# **Práctica 4.3: Redes neuronales convolucionales (CNN)**

<hr>

## **1. Introducción**

Las redes neuronales que hemos considerado hasta ahora siempre han trabajado con *datos estructurados*, es decir, datos que pueden almacenarse en tablas. Actualmente, es cada vez más común enfrentarse a problemas que requieren el manejo de **datos no estructurados**, como imágenes, texto o audio, siendo las imágenes el tipo de dato más frecuente. En este tipo de problemas, a diferencia de los que hemos visto hasta ahora, las entradas del modelo no son vectores de valores extraídos de un conjunto de datos, sino imágenes.

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> Las imágenes en escala de grises se codifican como <b>matrices</b> bidimensionales, mientras que las imágenes a color se representan mediante <b>tensores</b> tridimensionales.
</div>

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:1200px">
        <img src="https://i.imgur.com/urLXh6F.png">   
    </div>
</center>

Los modelos que hemos visto hasta ahora están pensados para trabajar con vectores, por lo que no pueden trabajar con las imágenes directamente. Si quisiesemos utilizar un modelo clásico (como regresión logística o SVM) con imágenes, sería necesario transformar la imagen en un vector. Una forma común de hacerlo es *aplanar* la imagen. Aplanar una imagen significa convertir la matriz 2D (o tensor 3D) en un vector unidimensional concatenando las filas (o canales de color) de la imagen una tras otra. 

Por ejemplo, una imagen de $28\times28$ píxeles se transformaría en un vector de $784$ elementos. Este vector resultante ya se podría utilizar como entrada para un modelo clásico. Aunque este método puede implicar la pérdida de información espacial relevante de la imagen original, representa una manera sencilla de adaptar modelos clásicos al procesamiento de imágenes. En redes tradicionales con capas densas (fully connected), como las que hemos visto hasta ahora, sería necesario realizar el mismo proceso. Esto implica que, incluso para imágenes pequeñas como la del ejemplo, se obtienen vectores de gran tamaño, lo que requiere aprender una gran cantidad de pesos o parámetros.

Buscando solucionar estos problemas surgen las **redes neuronales convolucionales (CNN)**, cuya arquitectura se compone de dos partes:

* **Extractor de características**: Esta parte se encarga de extraer las características relevantes de la imagen, es decir, aprende un vector de poca dimensión que representa a la imagen. Se compone de una serie de capas **convolucionales** y capas de **pooling**. 
* **Parte totalmente conectada**: Esta parte se encarga de resolver, a partir del vector aprendido, el problema que se pretende solventar (clasificación, regresión, ...). Está compuesta por una serie de capas densas (fully connected), como las redes utilizadas previamente.
<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:1200px">
        <img src="https://i.imgur.com/vsSJBcu.png">   
    </div>
</center>

<div class="alert alert-block alert-warning">
    <strong>Nota:</strong> La <i>parte convolucional</i> o el <i>extractor de carácteristicas</i> de una CNN es la encargada de aprender una <b>representación vectorial</b> de la imagen que luego será la entrada de la parte totalmente conectada encargada de realizar la predicción final.
</div>


### **Objetivo**
En esta práctica aprenderás a resolver un problema de clasificación de imágenes creando una red convolucional con `tensorFlow` y `keras`.

## **2. Configurar GPU**



Para poder acelerar el entrenamiento lo máximo posible, en esta práctica haremos uso de las *GPUs* instaladas en los ordenadores de prácticas.

Hasta ahora, todos los modelos que hemos entrenado se han ejecutado en *CPU* puesto que `tensorflow` ejecuta en este dispositivo por defecto. Esta librería, solo permite el uso de *GPU* en Windows mediante **Windows Subsystem for Linux (WSL)**.

Esta característica nos permite tener un environment Linux directamente en Windows sin la necesidad de crear una máquina virtual o configurar un arranque dual. Como verás, podremos ejecutar programas de línea de comandos de Linux directamente en Windows.

##### **Activar e instalar WSL**

Tendrás que abrir una ventana de comandos `cmd.exe` y ejecutar:

In [ ]:
wsl --install

* Indica tu **UO** como usuario y también como contraseña.

Dentro de la consola de Linux (Ubuntu por defecto) tendremos que instalar el `conda` y crear el environment de la asignatura.

##### **Instalar conda**

Descargamos `miniconda` (versión reducida de `conda`) e instalamos.

<div class="alert alert-block alert-warning">
    <strong>Tendrás que decir que si (Yes) a la pregunta <i>Do you wish to update your shell profile to automatically initialize conda?</i>.</strong>
</div>

In [ ]:
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -o Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh

* Cierra el terminal WSL y vuelve a abrirlo para que actualize las variables del entorno.

Puedes encontrarlo de nuevo en el menú de Windows buscando `WSL`.

A continuación creamos de nuevo el environment `SSII`.

In [ ]:
conda create --name "SSII" python=3.10
conda activate "SSII"

Instalamos dentro del environment las librerías necesarias, incluyendo la versión de `tensorflow` con soporte de GPU.

In [ ]:
pip install ipykernel pandas seaborn scikit-learn
pip install tensorflow[and-cuda]

##### **Visual Studio Code**

En este momento tenemos dos sistemas operativos, el Windows nativo y un Linux que se ejecuta dentro del mismo. Nuestro objetivo será ejecutar este Notebook dentro del Linux.

Para ello tendremos que indicar al Visual Studio Code, **que se conencte a una máquina diferente**. La forma más sencilla de hacerlo es:

* Abre una nueva ventana de VSCode (`File > New Window`).
* En la nueva ventana de VSCode, abajo a la izquierda aparece un icono de dos flechas enfrentadas.
* Al hacer click nos aparece un menú superior, en este seleccionamos <i>Connect to WSL</i>.
* Si todo funciona, Abajo a la izquierda aparecerá algo como **WSL: Ubuntu**.

Lo siguiente será instalar las **extensiones de VSCode**. Ya las habías instalado en Windows, pero ahora tendrás que hacerlo en la distribución Linux.

* Instala *Python* y *Jupyter*.

##### **Sistema de archivos**

Recuerda que ahora estamos dentro de otra máquina, por tanto tendremos que almacenar los notebooks de las prácticas dentro de la misma.

Windows nos simplifica mucho este acceso, si abres un explorador de archivos, simplemente tendrás que ir a la carpeta *Linux* que aparece en los accesos directos de la izquierda:

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:600px">
        <img src="https://i.imgur.com/dEqSKsg.png" style="height:400px">   
    </div>
</center>

* Crea la carpeta `SSII` en `Linux/Ubuntu/home/**tu UO**/`.
* Copia este notebook dentro de la misma.
* Abre esa carpeta desde VSCode (`File > Open Folder...`).
* Abre el notebook y selecciona como kernel el environment `conda` que acabas de crear.

La forma más sencilla de verificar que `tensorflow` tiene acceso a la GPU es ejecutar el siguiente código: 

In [ ]:
! export TF_CPP_MIN_LOG_LEVEL=2  # Para que tensorflow muestre solo advertencias y errores

import tensorflow as tf

seed = 2533

print("-"*50)
print("Num GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))
print("-"*50)

También puedes revisar el uso de GPU en el momento actual con el siguiente comando:

In [ ]:
! nvidia-smi

Si prefieres monitorizar el uso de GPU de forma constante, puedes abrir una terminal de WSL (desde Windows o desde el VScode en `Terminal > New terminal`) y ejecutar el siguiente comando:

In [ ]:
! watch -n1 nvidia-smi

Simplemente ejecuta el comando anterior cada 1 segundo. Para salir tendrás que pulsar `Ctrl+C`.

<hr>

## **3. Redes convolucionales**

Una vez tenemos todo configurado vamos a proponer un posible problema.

Imagina que trabajas para el servicio de informática de Correos, y que en la actualidad tienen a una persona encargada de leer manualmente los destinatarios de las cartas (escritos a mano) y ubicarlas, en función del código postal, en una caja u otra.

Como ves, este proceso es muy lento, por lo que proponen instalar una cámara que tome imágenes de cada una de las cartas y que automáticamente lea los códigos postales. 

Ya poseen imágenes de los códigos postales de las cartas, pero no saben como extraer los dígitos de cada imagen, por tanto te plantean el siguiente problema:

<div class="alert alert-block alert-success">
    <b>Crear un modelo que, dada una imagen de un número escrito a mano, sea capaz de reconocer de que número se trata.</b> 
</div>

### **3.1 Preprocesamiento de datos**

Para ayudarte, ya se han encargado de crear manualmente un dataset que contiene imágenes de dígitos individuales y etiquetas indicando el número que aparece en la imagen.

<center>
    <div style="border-radius:5px; padding:10px; background:white; max-width:1000px">
        <img src="https://i.imgur.com/cHlLRXB.png">   
    </div>
</center>

Puedes descargar el conjunto mediante el siguiente código:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Este dataset contiene $70000$ imágenes en escala de grises de $28\times28$ de números del $0$ a $10$ escritos a mano. 

Como puedes apreciar, ya viene dividido en $X$, $Y$ así como en entrenamiento ($60000$) y test ($10000$).


In [ ]:
print(x_train.shape, y_train.shape)

Para simplificar, vamos a quedarnos solo con 20.000 ejemplos aleatorios de test

In [ ]:
import numpy as np

np.random.seed(seed)
indices = np.random.choice(len(x_train), size=20000, replace=False)

# Obtener las muestras
x_train = x_train[indices]
y_train = y_train[indices]

print(x_train.shape, y_train.shape)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Normaliza los datos manualmente entre 0 y 1.
    <hr>
    Las clases del <code>scikit-learn</code> no funcionan en este caso, puesto que están pensadas para tratar con vectores. 
</div>

In [ ]:
# Tu código aquí

Vamos a crear un gráfico para ver uno de los ejemplos del conjunto de train.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5,5))
plt.imshow(x_train[0], cmap="Greys")
plt.title(f"Ejemplo de imagen del dígito {y_train[1]}")
plt.show()

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Visualiza la imagen 1523 del conjunto de test. Añade su etiqueta (clase) real en el título.
</div>

In [ ]:
# Tu código aquí

### **3.2 Aprendizaje automático**

Recuerda que para abordar este problema con modelos de aprendizaje clásico, necesitamos primero transformar las imágenes de $28\times28$ en vectores.

Creamos dos nuevas variables para mantener las imágenes originales y su versión "aplanada".

In [ ]:
x_train_vector = x_train.reshape(-1, 28 * 28)
x_test_vector = x_test.reshape(-1, 28 * 28)

print(x_train.shape)
print(x_train_vector.shape)

Una vez que los datos han sido adaptados, podemos intentar abordar el problema con modelos de aprendizaje automático clásicos.

Para ver que métrica es más adecuada, analizamos antes cuantas imágenes hay de cada número (clase).

In [ ]:
sns.countplot(x=y_train, hue=map(str, y_train))

En este caso las clases están bastante balanceadas, por lo que la `f1 macro` será suficiente. 

Ten en cuenta que los métodos que no son baselines tardarán algo de tiempo en entrenar, puesto que ahora cada ejemplo se compone de $784$ entradas.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from tabulate import tabulate
    
def evaluate_model(Y_test, preds_test, model_name, average="binary"):
    preds_test = (preds_test >= 0.5).astype(int)
    metrics = [
        ("Accuracy", accuracy_score(Y_test, preds_test)),
        ("F1", f1_score(Y_test,preds_test, average=average))
    ]
    
    print(f"Resultados para {model_name}:")
    print(tabulate(metrics, headers=["Métrica", "TEST"], tablefmt="rounded_outline"))
    print()
    
# Baseline Random
baseline_random = DummyClassifier(strategy="uniform")
baseline_random.fit(x_train_vector, y_train)
preds_test = baseline_random.predict(x_test_vector)
evaluate_model(y_test, preds_test, "Baseline Random", average="macro")

# Baseline Zero-R
baseline_zero = DummyClassifier(strategy="most_frequent")
baseline_zero.fit(x_train_vector, y_train)
preds_test = baseline_zero.predict(x_test_vector)
evaluate_model(y_test, preds_test, "Baseline Zero-R", average="macro")

# KNN
model_knn = KNeighborsClassifier()
model_knn.fit(x_train_vector, y_train)
preds_test = model_knn.predict(x_test_vector)
evaluate_model(y_test, preds_test, "KNN", average="macro")

# Árboles de Decisión
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train_vector, y_train)
preds_test = model_tree.predict(x_test_vector)
evaluate_model(y_test, preds_test, "Árbol de decisión", average="macro")

Esto nos retorna los siguientes resultados:

<center>

| Modelo            | Accuracy (Test) | F1 (Test) |
|-------------------|-----------------|-----------|
| Baseline Random   | 0.114           | 0.032     |
| Baseline Zero-R   | 0.114           | 0.020     |
| KNN               | 0.211           | 0.120     |
| Árbol de decisión | 0.202           | 0.112     |

</center>

### **3.3 Aprendizaje profundo**

Vamos a crear dos opciones, una *red totalmente conectada* que reciba el vector de $784$ números, y a continuación una *red convolucional* que reciba directamente las imágenes.

Antes de continuar necesitaremos codificar correctamente las salidas esperadas $Y$ de nuestro modelo.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Codifica y sobrescribe las Y utilizando one-hot.
</div>

In [ ]:
# Tu código aquí

##### **Red totalmente conectada**

Vamos a fijar previamente las semillas del `tensorflow` y crear de nuevo la función auxiliar `plot_loss_history()` para ver la evolución de la loss durante el entrenamiento.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, random

# Fijar las semillas de las librerías para que los resultados se repitan.
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

def plot_loss_history(history):
    # Extraer los datos del historial
    loss = history.history['loss']
    val_loss = history.history.get('val_loss', None)  # Puede no existir si no se usó validación
    epochs = range(1, len(loss) + 1)

    # Crear un DataFrame para seaborn
    data = pd.DataFrame({ 'Epoch': list(epochs) * 2, 'Loss': loss + (val_loss if val_loss else []), 'Type': ['Train'] * len(loss) + (['Validation'] * len(val_loss) if val_loss else []) })

    # Crear el gráfico
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=data, x="Epoch", y="Loss", hue="Type")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Evolución de la loss durante el entrenamiento")
    plt.legend(title="Conjunto")
    plt.grid(True)
    plt.show()


<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea una <i>red totalmente contectada</i> para resolver este problema y rellena la tabla. Ajusta los hiperparámetros si lo deseas.
</div>


<center>

| Modelo            | Accuracy (Test) | F1 (Test) |
|-------------------|-----------------|-----------|
| Baseline Random   | 0.114           | 0.032     |
| Baseline Zero-R   | 0.114           | 0.020     |
| KNN               | 0.211           | 0.120     |
| Árbol de decisión | 0.202           | 0.112     |
| Red Neuronal      |                 |           |

</center>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

def red_fully_connected(learning_rate):

    model = Sequential()

    # Tu código aquí

    optim = Adam(learning_rate=learning_rate)
    model.compile(loss='', optimizer=optim)

    return model

# Creamos la red desde cero
model_fcn = red_fully_connected(learning_rate = 0.0005)

# Ver el summary
model_fcn.summary()

# Entrenamos
history = model_fcn.fit(x_train_vector, y_train, validation_split=0.2, batch_size=256, epochs=20, verbose=2)

# Visualizamos
plot_loss_history(history)

# Evaluar en test
preds_test = model_fcn.predict(x_test_vector)
evaluate_model(y_test, preds_test, "Red neuronal", average="macro")

##### **Red convolucional**

A continuación se te proporciona la arquitectura de una red convolucional pensada para resolver este problema.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Completa el <i>tamaño de la entrada</i>, la <i>función de activación de la última capa</i> y la <i>función de pérdida</i>.
</div>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

def red_convolucional(learning_rate):
    model = Sequential()
    model.add(Input(shape=()))  # Entrada como imagen en escala de grises

    # EXTRACTOR DE CARACTERÍSTICAS -------------------------------------------
    # Primer bloque de convolución y max pooling
    model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Segundo bloque de convolución y max pooling
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Tercer bloque de convolución y max pooling
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # PARTE TOTALMENTE CONECTADA ---------------------------------------------
    # Aplanar y capas densas finales
    model.add(Flatten()) # Este vector es el que aprende la red durante el entrenamiento y le sirve para representar a cada imagen
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='', name="output_layer"))

    optim = Adam(learning_rate=learning_rate)
    model.compile(loss='', optimizer=optim)

    return model

# Creamos la red desde cero
model_cnn = red_convolucional(learning_rate = 0.0005)

# Ver el summary
model_cnn.summary()

Finalmente entrenamos y evaluamos en test.

In [ ]:
# Entrenamos
history = model_cnn.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=20, verbose=2)

# Visualizamos
plot_loss_history(history)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Añade el código necesario para evaluar tu modelo y rellena la tabla.
</div>

<center>

| Modelo            | Accuracy (Test) | F1 (Test) |
|-------------------|-----------------|-----------|
| Baseline Random   | 0.114           | 0.032     |
| Baseline Zero-R   | 0.114           | 0.020     |
| KNN               | 0.211           | 0.120     |
| Árbol de decisión | 0.202           | 0.112     |
| Red Neuronal      |                 |           |
| Red Convolucional |                 |           |

</center>

In [ ]:
# Tu código aquí

<hr>

## **4. Ejercicios**


<div class="alert alert-block alert-success">
    <b>Crear un modelo que dada la imagen de un dígito, prediga si <u>es un cuatro o un nueve.</u></b> Ya se proporciona el código encargado de crear el dataset necesario.
</div>

In [ ]:
import numpy as np
import tensorflow as tf

# Cargamos el dataset MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

def filter_and_relabel(x, y, class_pos=1, class_neg=7):
    # Filtramos solo los dígitos deseados
    idx = np.where((y == class_pos) | (y == class_neg))[0]
    x_filtered = x[idx]
    y_filtered = y[idx]

    # Etiquetamos: 1 si es class_pos, 0 si es class_neg
    y_binary = (y_filtered == class_pos).astype(np.uint8)

    # Balanceamos (debería estar casi balanceado ya)
    idx_pos = np.where(y_binary == 1)[0]
    idx_neg = np.where(y_binary == 0)[0]

    n = min(len(idx_pos), len(idx_neg))
    np.random.seed(42)
    idx_pos_sampled = np.random.choice(idx_pos, size=n, replace=False)
    idx_neg_sampled = np.random.choice(idx_neg, size=n, replace=False)

    idx_total = np.concatenate([idx_pos_sampled, idx_neg_sampled])
    np.random.shuffle(idx_total)

    return x_filtered[idx_total], y_binary[idx_total]

# Aplicamos para train y test
x_train, y_train = filter_and_relabel(x_train, y_train, class_pos=4, class_neg=9)
x_test, y_test = filter_and_relabel(x_test, y_test, class_pos=4, class_neg=9)

In [ ]:
# Tu código aquí